In [1]:
import pandas as pd
import json

In [2]:
with open('../data/bedca_raw.json', 'r') as fp:
    food_dict = json.load(fp)

FileNotFoundError: [Errno 2] No such file or directory: '../data/bedca.json'

In [4]:
#formato inicial
food_dict['Arroz con leche']

[['alcohol (etanol)', '0', 'g'],
 ['energía, total', '383 (92)\n\t\t\t\t\t\t\t', 'kJ (kcal)'],
 ['grasa, total (lipidos totales)', '1.3', 'g'],
 ['proteina, total', '3.3', 'g'],
 ['agua (humedad)', '78.9', 'g'],
 ['fibra, dietetica total', '0.1', 'g'],
 ['carbohidratos', '16.4', 'g'],
 ['ácido graso 22:6 n-3 (ácido docosahexaenóico)', '-', '-'],
 ['ácidos grasos, monoinsaturados totales', '0.31', 'g'],
 ['ácidos grasos, poliinsaturados totales', '0.08', 'g'],
 ['ácidos grasos saturados totales', '0.77', 'g'],
 ['ácido graso 12:0 (láurico)', '-', '-'],
 ['ácido graso 14:0 (ácido mirístico)', '-', '-'],
 ['ácido graso 16:0 (ácido palmítico)', '-', '-'],
 ['ácido graso 18:0 (ácido esteárico)', '-', '-'],
 ['ácido graso 18:1 n-9 cis (ácido oléico)', '-', '-'],
 ['colesterol', '9', 'mg'],
 ['ácido graso 18:2', '-', '-'],
 ['ácido graso 18:3', '-', '-'],
 ['ácido graso 20:4 n-6  (ácido araquidónico)', '-', '-'],
 ['ácido graso 20:5 (ácido eicosapentaenóico)', '-', '-'],
 ['Vitamina A equival

In [5]:
multipliers = {'g': 1, 'mg': 1e3, 'ug': 1e6}

def preprocess_bedca(name,values):
    '''
    transforma el diccionario en una lista (formato csv) pasando el valor a gramos si es necesario (kcal para la enegría)
    '''
    new_bedca = []
    new_bedca.append(name)
    new_bedca.append(values[-1])
    for value in values[:-1]:
        if value[0] == 'energía, total':
            new_bedca.append(getValueInKcal(value[1]))
        else:
            try:
                new_bedca.append(getValueInGrams(value[1],value[2]))
            except ValueError:
                print(name,values)
    return new_bedca

def getValueInGrams(value,unit):
    if value == 'traza':
        return 1e-12 #valor muy pequeño para detectar alergias
    elif value == '-' or value == '':
        return None
    else:
        return float(value)/multipliers[unit] #resultado en gramos

def getValueInKcal(value):
    return value[value.find('(')+1 : value.find(')')]

In [6]:
food_list = []
for name,v in food_dict.items():
    if(len(v)==41): #descartados a los que les falte alguna columna
        food_list.append(preprocess_bedca(name,v))

In [7]:
len(food_list)

935

In [8]:
def getColumns():
    columns = ['nombre','categoria']
    nutri_values = food_dict['Aceite de algodón'][:-1]
    for value in nutri_values:
        columns.append(value[0])
    return columns

In [9]:
#NOMBRES DE LAS COLUMNAS
getColumns()

['nombre',
 'categoria',
 'alcohol (etanol)',
 'energía, total',
 'grasa, total (lipidos totales)',
 'proteina, total',
 'agua (humedad)',
 'fibra, dietetica total',
 'carbohidratos',
 'ácido graso 22:6 n-3 (ácido docosahexaenóico)',
 'ácidos grasos, monoinsaturados totales',
 'ácidos grasos, poliinsaturados totales',
 'ácidos grasos saturados totales',
 'ácido graso 12:0 (láurico)',
 'ácido graso 14:0 (ácido mirístico)',
 'ácido graso 16:0 (ácido palmítico)',
 'ácido graso 18:0 (ácido esteárico)',
 'ácido graso 18:1 n-9 cis (ácido oléico)',
 'colesterol',
 'ácido graso 18:2',
 'ácido graso 18:3',
 'ácido graso 20:4 n-6  (ácido araquidónico)',
 'ácido graso 20:5 (ácido eicosapentaenóico)',
 'Vitamina A equivalentes de retinol de actividades de retinos y carotenoides',
 'Vitamina D',
 'Viamina E equivalentes de alfa tocoferol de actividades de vitámeros E',
 'folato, total',
 'equivalentes de niacina, totales',
 'riboflavina',
 'tiamina',
 'Vitamina B-12',
 'Vitamina B-6, Total',
 'Vita

In [10]:
df = pd.DataFrame(food_list,columns=getColumns())

In [11]:
df.head()


,nombre,categoria,alcohol (etanol),"energía, total","grasa, total (lipidos totales)","proteina, total",agua (humedad),"fibra, dietetica total",carbohidratos,ácido graso 22:6 n-3 (ácido docosahexaenóico),...,Vitamina C (ácido ascórbico),calcio,"hierro, total",potasio,magnesio,sodio,fósforo,ioduro,"selenio, total",zinc (cinc)
0,Arroz con leche,Lácteos,0.0,92,1.30,3.30,78.90,0.1,16.40,NaN,...,0.00000,0.109,0.00020,0.138,0.01400,0.038,0.096,0.000006,0.000002,0.00030
1,Batido de chocolate,Lácteos,0.0,83,3.60,2.70,83.10,0.8,9.80,NaN,...,0.00100,0.111,0.00030,0.192,0.02200,0.049,0.095,0.000007,0.000002,0.00040
2,Batido de fresa,Lácteos,0.0,66,1.00,2.80,84.20,0.8,11.20,NaN,...,0.00009,0.110,0.00009,0.183,0.01131,0.095,0.090,0.000007,0.000002,0.00039
3,Batido de soja,Lácteos,0.0,82,5.20,5.00,85.90,0.0,3.90,NaN,...,0.00047,0.100,0.00040,0.120,0.01500,0.032,0.047,0.000000,0.000001,0.00020
4,Batido fermentado de soja,Lácteos,0.0,90,1.18,2.35,78.18,0.6,17.06,NaN,...,0.01760,0.176,0.00064,0.120,0.01500,0.015,0.047,0.000000,0.000001,0.00020


In [12]:
#VALORES NULOS, tal vez hace falta rellenar o eliminar filas
df.isna().sum()

nombre                                                                           0
categoria                                                                        0
alcohol (etanol)                                                                 0
energía, total                                                                   0
grasa, total (lipidos totales)                                                   0
proteina, total                                                                  0
agua (humedad)                                                                   2
fibra, dietetica total                                                          10
carbohidratos                                                                    2
ácido graso 22:6 n-3 (ácido docosahexaenóico)                                  829
ácidos grasos, monoinsaturados totales                                          41
ácidos grasos, poliinsaturados totales                                          40
ácid

In [13]:
df.to_csv('../data/bedca.csv')